In [1]:
import torch
from torch import nn
import pickle as pkl
import numpy as np
import MinkowskiEngine as ME

In [2]:
filehandler = open("data.pkl","rb")
dataset = pkl.load(filehandler)
filehandler.close()

In [3]:
torch.cuda.is_available()

True

In [4]:
dev = torch.device("cuda")
dev

device(type='cuda')

In [9]:
class Conv3DFeatureDetector(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        size_1 = 100
        size_2 = 200
        self.conv1 = nn.Sequential(
            ME.MinkowskiConvolution(1, size_1, 
                      kernel_size=(kernel_size),
                      dimension=3,
                                   ),
            ME.MinkowskiTanh(),
#             nn.MaxPool3d((2, 2, 2)),
        )
        self.fc1 = ME.MinkowskiLinear(size_1, size_2)
        self.relu = ME.MinkowskiTanh()
        self.fc2 = ME.MinkowskiLinear(size_2, 1)
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = ME.MinkowskiFunctional.sigmoid(out)
        return out
    
# class Conv3DFeatureDetector(nn.Module):
#     def __init__(self, kernel_size):
#         super().__init__()
        
#         size_1 = 100
#         size_2 = 200
#         self.conv1 = nn.Sequential(
#             nn.Conv3d(1, size_1, kernel_size=(kernel_size, kernel_size, kernel_size), padding= (kernel_size - 1)//2),
#             nn.Tanh(),
# #             nn.MaxPool3d((2, 2, 2)),
#         )
#         self.fc1 = nn.Linear(size_1, size_2)
#         self.relu = nn.Tanh()
#         self.fc2 = nn.Linear(size_2, 1)
        
#     def forward(self, x):
#         out = self.conv1(x)
#         out = out.permute(0,2,3,4,1)
#         out = self.fc1(out)
#         out = self.relu(out)
#         out = self.fc2(out)
#         out = torch.sigmoid(out)
#         return out

In [41]:
X = np.vstack([x[None,None,:,:,:] for (x,y) in dataset])
X = X.astype(np.float32)
X = torch.from_numpy(X)

Y = np.vstack([y[None,:,:,:,None] for (x,y) in dataset])
Y = Y.astype(np.float32)
Y = torch.from_numpy(Y)
print(X.shape)
print(Y.shape)

X_train = X[:500]
Y_train = Y[:500]

def get_sparse_tensor_from_numpy(X, d=None):
    idxs = torch.where(X)
    num_nonzero_elements = idxs[0].shape[0]
    num_dimensions = len(idxs)
    coords = torch.vstack(idxs)
    coords = coords.type(torch.int32)
    coords = torch.transpose(coords, 0,1).contiguous()
    feat = torch.ones(coords.shape[0],1)
    if d is None:
        X = ME.SparseTensor(feat, coordinates=coords)
    else:
        X = ME.SparseTensor(feat, coordinates=coords, device=d)
    
    return X


torch.Size([1000, 1, 35, 35, 32])
torch.Size([1000, 35, 35, 32, 1])


In [42]:
X_train.mean()

tensor(0.0345)

In [43]:
# X.cuda()
model = Conv3DFeatureDetector(9)
model.to(dev)



Conv3DFeatureDetector(
  (conv1): Sequential(
    (0): MinkowskiConvolution(in=1, out=100, kernel_size=[9, 9, 9], stride=[1, 1, 1], dilation=[1, 1, 1])
    (1): MinkowskiTanh()
  )
  (fc1): MinkowskiLinear(in_features=100, out_features=200, bias=True)
  (relu): MinkowskiTanh()
  (fc2): MinkowskiLinear(in_features=200, out_features=1, bias=True)
)

In [44]:
learning_rate = 1.0
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [45]:
X_train[:10,0,:,:,:].shape

torch.Size([10, 35, 35, 32])

In [46]:
batch_size = 10
output = None
batch_y = None
batch_x = None
for epoch in range(1000):
#     permutation = torch.randperm(X_torch.size()[0])
    permutation = torch.arange(X_train.size()[0])
    
    total_loss = 0.0
    for i in range(0,X_train.size()[0], batch_size):
#         print(i)
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = X_train[indices], Y_train[indices]
        batch_y = batch_y.permute(0,4,1,2,3)
        batch_x = get_sparse_tensor_from_numpy(batch_x[:,0,:,:,:],d=dev)
        
        # in case you wanted a semi-full example
        outputs = model.forward(batch_x)
        
        outputs = outputs.dense(shape=batch_y.shape)[0].cpu()
        
        loss = (outputs - batch_y)**2
        loss[batch_y == 1.0] *= 100.0
        loss = loss.mean()
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    print("total loss: ", total_loss)

total loss:  11.682123735547066
total loss:  11.50080032646656
total loss:  11.21671311557293
total loss:  11.03125473856926
total loss:  10.950746968388557
total loss:  10.907248809933662
total loss:  10.87880676984787
total loss:  10.858369797468185
total loss:  10.842954576015472
total loss:  10.830983027815819
total loss:  10.821486189961433
total loss:  10.81381580233574
total loss:  10.807520359754562
total loss:  10.802278146147728
total loss:  10.797856554389
total loss:  10.794084787368774
total loss:  10.790835857391357
total loss:  10.788012146949768
total loss:  10.78553955256939
total loss:  10.78335927426815
total loss:  10.781424343585968
total loss:  10.779697820544243
total loss:  10.77814881503582
total loss:  10.776752784848213
total loss:  10.775488585233688
total loss:  10.774339467287064
total loss:  10.773290857672691
total loss:  10.772330582141876
total loss:  10.771448493003845
total loss:  10.770635426044464
total loss:  10.769883766770363
total loss:  10.769

KeyboardInterrupt: 

In [47]:
X_test = X[-50:]
Y_test = Y[-50:]

In [48]:
model_cpu = model.cpu()

In [49]:
pred_Y = model.forward(get_sparse_tensor_from_numpy(X_test[:,0,:,:,:]))

In [50]:
permuted_Y_test = Y_test.permute(0,4,1,2,3)
pred_Y = pred_Y.dense(shape=permuted_Y_test.shape)[0]
pred_Y = pred_Y.permute(0,2,3,4,1)
pred_Y.shape

torch.Size([50, 35, 35, 32, 1])

In [51]:
pred_Y[Y_test[-100:] == 1.0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<IndexBackward0>)

In [52]:
pred_Y[Y_test[-100:] == 0.0]

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<IndexBackward0>)

In [53]:
with open("result.pkl","wb") as f:
    pkl.dump((X_test.cpu().detach().numpy(), pred_Y.cpu().detach().numpy()), f)